In [21]:
from src.util import connect_to_db
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

level_modifiers = {
    0: 0,
    1: 1/105,
    2: 250/105,
    3: 595/105,
    4: 1414/105,
    5: 3360/105,
    6: 8000/105,
    7: 19000/105,
    8: 45255/105,
    9: 107700/105,
    10: 256000/105    
}

def records_to_df(records):
    return pd.DataFrame([dict(r) for r in records])

conn = await connect_to_db()






In [22]:
contracts = records_to_df(await conn.fetch("SELECT * FROM esi.contracts"))
contract_items = records_to_df(await conn.fetch("SELECT * FROM esi.contract_items"))
type_ids = records_to_df(await conn.fetch("SELECT * FROM sde.type_ids"))
blueprints = records_to_df(await conn.fetch("SELECT * FROM sde.blueprints"))
market_aggregates = records_to_df(await conn.fetch("SELECT * FROM market.aggregates WHERE location_id = 60003760"))

In [23]:
contracts = records_to_df(await conn.fetch("SELECT * FROM esi.contracts"))
contract_items = records_to_df(await conn.fetch("SELECT * FROM esi.contract_items"))
type_ids = records_to_df(await conn.fetch("SELECT * FROM sde.type_ids"))
blueprints = records_to_df(await conn.fetch("SELECT * FROM sde.blueprints"))

contracts = contracts[contracts["contract_id"].map(contract_items.groupby("contract_id").size()) == 1]
contracts = contracts.merge(contract_items, on="contract_id")
contracts = contracts[(contracts["type"] == 'item_exchange') & (contracts["is_included"]) & (contracts["material_efficiency"].notna()) & (~contracts["is_blueprint_copy"])]
contracts = contracts.merge(type_ids, left_on="type_id", right_on="type_id")
contracts = contracts.merge(blueprints, left_on="type_id", right_on="blueprint_type_id")

contracts["premium"] = (contracts["price"] - contracts["base_price"]).map(float)
contracts["pct_premium"] = (contracts["price"] / contracts["base_price"]).map(float)
contracts["research_time"] = contracts["time_efficiency"].map(lambda x: level_modifiers[int(x/2)]) * contracts["research_time_time"] \
    + contracts["material_efficiency"].map(lambda x: level_modifiers[int(x)]) * contracts["research_material_time"]


contracts["research_hours"] = contracts["research_time"] / 3600
contracts[["contract_id", "name", "base_price", "price", "material_efficiency", "time_efficiency", "pct_premium", "research_hours", "region_id"]].sort_values("pct_premium", ascending=True).head(40)

,contract_id,name,base_price,price,material_efficiency,time_efficiency,pct_premium,research_hours,region_id
1742,201545077,Complex Asteroid Mining Crystal Type A I Bluep...,7200000.00,5000000.00,10.0,20.0,0.694444,142.222222,10000002
722,201545043,Coherent Asteroid Mining Crystal Type A I Blue...,3600000.00,2700000.00,10.0,20.0,0.750000,142.222222,10000002
718,201484341,Coherent Asteroid Mining Crystal Type A I Blue...,3600000.00,2750000.00,10.0,20.0,0.763889,142.222222,10000002
719,201484349,Coherent Asteroid Mining Crystal Type A I Blue...,3600000.00,2750000.00,10.0,20.0,0.763889,142.222222,10000002
1739,201508954,Complex Asteroid Mining Crystal Type A I Bluep...,7200000.00,5500000.00,10.0,20.0,0.763889,142.222222,10000002
1740,201508960,Complex Asteroid Mining Crystal Type A I Bluep...,7200000.00,5500000.00,10.0,20.0,0.763889,142.222222,10000002
717,201484338,Coherent Asteroid Mining Crystal Type A I Blue...,3600000.00,2750000.00,10.0,20.0,0.763889,142.222222,10000002
3812,201615911,Cap Booster 200 Blueprint,2500000.00,1999999.00,0.0,0.0,0.800000,0.000000,10000002
990,201620511,Simple Asteroid Mining Crystal Type A I Blueprint,2000000.00,1600000.00,10.0,20.0,0.800000,142.222222,10000043
3252,201010509,Small Artillery Battery Blueprint,25000000.00,20000000.00,0.0,0.0,0.800000,0.000000,10000002


In [24]:
# T2 BPO contracts
contracts = records_to_df(await conn.fetch("SELECT * FROM esi.contracts"))
contract_items = records_to_df(await conn.fetch("SELECT * FROM esi.contract_items"))
contract_items = contract_items.merge(type_ids, left_on="type_id", right_on="type_id")
contract_items = contract_items.merge(contracts, left_on="contract_id", right_on="contract_id")
contract_items = contract_items[(contract_items["meta_group_id"] == 2.0) & (contract_items["is_blueprint_copy"] == False) & (contract_items["material_efficiency"].notna())]
contract_items[["name", "price", "region_id", "type"]]

,name,price,region_id,type
302530,Dual Heavy Beam Laser II Blueprint,250000000000.00,10000002,item_exchange
